In [1]:
import os

In [2]:
%pwd

'c:\\Users\\csawa\\Desktop\\Chest_X-Ray-Pneumonia_Classification\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\csawa\\Desktop\\Chest_X-Ray-Pneumonia_Classification'

In [28]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

In [32]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [33]:
import os 
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")


        except Exception as e:
            raise e
        

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [34]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-06 21:21:49,279: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-06 21:21:49,282: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-06 21:21:49,284: INFO: 960570527: Downloading data from https://drive.google.com/file/d/1mLkqw7CjXpnF5pAJG4WD22OpHCH7sRFA/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1mLkqw7CjXpnF5pAJG4WD22OpHCH7sRFA
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1mLkqw7CjXpnF5pAJG4WD22OpHCH7sRFA&confirm=t&uuid=b7ba93a2-1b93-4460-894d-76443030baae
To: c:\Users\csawa\Desktop\Chest_X-Ray-Pneumonia_Classification\artifacts\data_ingestion\data.zip
100%|██████████| 2.46G/2.46G [04:23<00:00, 9.36MB/s]

[2024-05-06 21:26:14,829: INFO: 960570527: Downloaded data from https://drive.google.com/file/d/1mLkqw7CjXpnF5pAJG4WD22OpHCH7sRFA/view?usp=sharing into file artifacts/data_ingestion/data.zip]



  0%|          | 0.00/2.46G [16:24<?, ?B/s]
